In [ ]:
import sys
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from datasets import load_dataset

os.environ["TOKENIZERS_PARALLELISM"] = "false"

print("✅ Import thành công!")

In [ ]:
!pip install -U huggingface-hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# ========== HUẤN LUYỆN SFT (Supervised Fine-Tuning) ==========
from trl import SFTConfig

model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
dataset_file = "dataset.jsonl"
output_dir = "./results_reframebot_llama3"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

lora_config = LoraConfig(
    r=6,
    lora_alpha=12,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

sft_config = SFTConfig(
    output_dir=output_dir,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    optim="paged_adamw_8bit",
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=50,
    save_steps=100,
    fp16=False,
    bf16=True,
    gradient_checkpointing=True,
    report_to="tensorboard",
    logging_first_step=True,
    max_length=384,
    packing=False
)

print(f"Đang tải model: {model_name}...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0},
)

dataset = load_dataset("json", data_files=dataset_file, split="train")
print(f"Dataset: {len(dataset)} mẫu")

def formatting_func(example):
    return tokenizer.apply_chat_template(
        example['messages'],
        tokenize=False,
        add_generation_prompt=False
    )

trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    formatting_func=formatting_func,
    train_dataset=dataset,
    peft_config=lora_config,
    args=sft_config
)

print("--- BẮT ĐẦU HUẤN LUYỆN SFT ---")
trainer.train()
print("--- HOÀN TẤT ---")

final_adapter_path = os.path.join(output_dir, "final_checkpoint")
trainer.save_model(final_adapter_path)
print(f"Đã lưu: {final_adapter_path}")

In [ ]:
# ========== HUẤN LUYỆN DPO (Direct Preference Optimization) ==========
from trl import DPOTrainer, DPOConfig

base_model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
adapter_to_tune_path = "D:\\Work\\AI\\Prototype_5\\checkpoint-423"
dpo_dataset_file = "data/dataset_dpo.jsonl"
dpo_output_dir = "./results_reframebot_DPO"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

print("Đang tải base model + adapter SFT...")
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map={"": 0},
)

model = PeftModel.from_pretrained(base_model, adapter_to_tune_path)
print("Model đã sẵn sàng")

dpo_config = DPOConfig(
    output_dir=dpo_output_dir,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=24,
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",
    learning_rate=5e-6,
    num_train_epochs=3,
    bf16=True,
    logging_steps=2,
    save_steps=2,
    report_to="tensorboard",
    beta=0.1,
    max_length=512,
    max_prompt_length=512,
)

dpo_dataset = load_dataset("json", data_files=dpo_dataset_file, split="train")
dpo_dataset = dpo_dataset.shuffle(seed=42)
print(f"Dataset DPO: {len(dpo_dataset)} mẫu")

dpo_trainer = DPOTrainer(
    model,
    ref_model=None,
    args=dpo_config,
    train_dataset=dpo_dataset,
    processing_class=tokenizer,
    peft_config=lora_config
)

print("--- BẮT ĐẦU HUẤN LUYỆN DPO ---")
dpo_trainer.train(resume_from_checkpoint=True)
print("--- HOÀN TẤT ---")

final_dpo_path = os.path.join(dpo_output_dir, "final_dpo_checkpoint")
dpo_trainer.save_model(final_dpo_path)
print(f"Đã lưu: {final_dpo_path}")

In [ ]:
# ========== HUẤN LUYỆN GUARDRAIL (Classifier) ==========
from transformers import AutoModelForSequenceClassification, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

MODEL_NAME = "distilbert-base-uncased"
DATA_FILE = "data/guardrail_dataset.jsonl"
OUTPUT_DIR = "./guardrail_model"

id2label = {0: "TASK_1", 1: "TASK_2", 2: "TASK_3"}
label2id = {"TASK_1": 0, "TASK_2": 1, "TASK_3": 2}

dataset = load_dataset("json", data_files=DATA_FILE, split="train")
dataset = dataset.shuffle(seed=42)
dataset_dict = dataset.train_test_split(test_size=0.1, seed=42)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True, max_length=128)

tokenized_datasets = dataset_dict.map(preprocess_function, batched=True)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,
    id2label=id2label,
    label2id=label2id
)

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='macro')
    return {'accuracy': accuracy, 'f1': f1, 'precision': precision, 'recall': recall}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

print("--- BẮT ĐẦU HUẤN LUYỆN GUARDRAIL ---")
trainer.train()
print("--- HOÀN TẤT ---")

trainer.save_model(OUTPUT_DIR)
print(f"Đã lưu: {OUTPUT_DIR}")